In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif, RFE, VarianceThreshold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from pyswarms.discrete import BinaryPSO
import time

print('done - imports')

# Load and preprocess
df = pd.read_csv('SWaT.csv')
df.columns = df.columns.str.strip()
df['Normal/Attack'] = (df['Normal/Attack'] == 'Attack').astype(int)
X_raw = df.drop(['Timestamp', 'Normal/Attack'], axis=1)
y = df['Normal/Attack']

scaler = MinMaxScaler()
X_raw = pd.DataFrame(scaler.fit_transform(X_raw), columns=X_raw.columns)
X_train, X_test, y_train, y_test = train_test_split(X_raw, y, test_size=0.2, stratify=y, random_state=42)
print('done - preprocessing and split')

# PSO options
options = {'c1': 1.5, 'c2': 1.5, 'w': 0.8, 'k': 2, 'p': 2}
results = {}

# Helper function for PSO
def run_pso_on_features(X_tr, X_te, y_tr, y_te, method_name):
    start = time.time()
    X_tr_np = X_tr.values if isinstance(X_tr, pd.DataFrame) else X_tr
    X_te_np = X_te.values if isinstance(X_te, pd.DataFrame) else X_te
    
    def fitness(mask):
        scores = []
        for particle in mask:
            cols = [i for i, bit in enumerate(particle) if bit]
            if not cols:
                scores.append(0)
                continue
            clf = RandomForestClassifier(n_estimators=30, random_state=42)
            clf.fit(X_tr_np[:, cols], y_tr)
            preds = clf.predict(X_te_np[:, cols])
            scores.append(f1_score(y_te, preds, average='weighted'))
        return -1 * np.array(scores)
    
    n_feat = X_tr_np.shape[1]
    pso = BinaryPSO(n_particles=10, dimensions=n_feat, options=options)
    cost, pos = pso.optimize(fitness, iters=10)
    
    selected_indices = [i for i, bit in enumerate(pos) if bit]
    clf = RandomForestClassifier(n_estimators=30, random_state=42)
    clf.fit(X_tr_np[:, selected_indices], y_tr)
    preds = clf.predict(X_te_np[:, selected_indices])
    
    return {
        'num_features': len(selected_indices),
        'selected_indices': selected_indices,
        'f1': f1_score(y_te, preds, average='weighted'),
        'accuracy': accuracy_score(y_te, preds),
        'precision': precision_score(y_te, preds, average='weighted'),
        'recall': recall_score(y_te, preds, average='weighted'),
        'time': time.time() - start
    }

done - imports
done - preprocessing and split


In [2]:
# 7. Correlation-based + PSO
print('Running: Correlation-based + PSO...')
print('  - Calculating feature correlations with target...')
corr_matrix = X_train.corrwith(y_train).abs()
print('  - Selecting top 10 features by correlation...')
top_10_corr = corr_matrix.nlargest(10).index
X_train_corr = X_train[top_10_corr]
X_test_corr = X_test[top_10_corr]
print('  - Starting PSO optimization...')
results['Correlation_PSO'] = run_pso_on_features(X_train_corr, X_test_corr, y_train, y_test, 'Corr+PSO')
print(f"done - Corr+PSO: {results['Correlation_PSO']['num_features']} features, F1={results['Correlation_PSO']['f1']:.4f}")

Running: Correlation-based + PSO...
  - Calculating feature correlations with target...


C:\Users\chand\AppData\Roaming\Python\Python313\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\Users\chand\AppData\Roaming\Python\Python313\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
2025-11-19 13:31:54,199 - pyswarms.discrete.binary - INFO - Optimize for 10 iters with {'c1': 1.5, 'c2': 1.5, 'w': 0.8, 'k': 2, 'p': 2}


  - Selecting top 10 features by correlation...
  - Starting PSO optimization...


pyswarms.discrete.binary: 100%|██████████|10/10, best_cost=-0.984
2025-11-19 13:53:45,998 - pyswarms.discrete.binary - INFO - Optimization finished | best cost: -0.9836019014711483, best pos: [0 1 1 1 0 1 1 1 0 0]


done - Corr+PSO: 6 features, F1=0.9836


In [4]:
# If best_pos is the binary mask shown
best_pos = [0, 1, 1, 1, 0, 1, 1, 1, 0, 0]  # (copy directly as seen in the output)

# Or, if it is returned inside results['Correlation_PSO'] as e.g. 'best_pos'
# best_pos = results['Correlation_PSO']['best_pos']

selected_features = X_train_corr.columns[np.array(best_pos, dtype=bool)]
print("Selected features:", selected_features.tolist())

Selected features: ['FIT504', 'FIT503', 'UV401', 'PIT501', 'FIT501', 'PIT503']
